In [1]:
pip install langchain PyMuPDF faiss-cpu langchain_community tiktoken sentence-transformers PyPDF2 ollama pypdf 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install pypdf


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install --upgrade pypdf

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pypdf in c:\users\jy901\appdata\local\programs\python\python311\lib\site-packages (4.2.0)




[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
import time
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores.utils import DistanceStrategy
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain

# 시작 시간 기록
start = time.time()

print("Starting the PDF loading process...")
# PDF 로드
pdf_filepath = 'test_samsung.pdf'
loader = PyPDFLoader(pdf_filepath)
pages = loader.load()
print("PDF loaded successfully.")



print("Starting the text splitting process...")
# 텍스트 분할
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=150,  # 300
    chunk_overlap=40,  # 50
    encoding_name='cl100k_base'
)

chunks = text_splitter.split_documents(pages)
print("Text splitting completed.")
print("=====chunks=====")

print("Starting the embedding and vector store creation process...")
# 임베딩 생성 및 벡터 스토어 생성
embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-multitask',
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True},
)

vectorstore = FAISS.from_documents(chunks,
                                   embedding=embeddings_model,
                                   distance_strategy=DistanceStrategy.COSINE
                                  )

MY_FAISS_INDEX = "./MY_FAISS_INDEX_llama3"

if not os.path.exists(MY_FAISS_INDEX):
    os.makedirs(MY_FAISS_INDEX)
vectorstore.save_local(MY_FAISS_INDEX)
print(f"Vectorstore saved at {MY_FAISS_INDEX}")

vectorstore = FAISS.load_local(MY_FAISS_INDEX, embeddings_model, allow_dangerous_deserialization=True)
print("=====vectorstore loaded=====")

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

query = '요약 손익 계산서 테이블에서 22Q3의 총 매출액이 얼마야?'
docs = retriever.get_relevant_documents(query)

for doc in docs:
    print(doc.page_content)

# 문서 포맷팅
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

formatted_docs = format_docs(docs)

# 포맷팅된 문서와 질문 출력 => 잘나옴
print("===== formatted docs check =====")
print("Formatted Docs: ", formatted_docs)
print("Question: ", query)


llm = ChatOllama(model="Llama-3-Open-Ko-8B-Q8_0:latest")
system_template = '''
    너는 사실만을 말하는 금융 전문가야.
    질문이 주어진 문서의 내용과 관련이 있다면, 해당 문서를 바탕으로 구체적이고 명확한 답변을 해줘.
    모든 문서의 내용을 이해하고 고려해서 질문과 관련된 정보를 모두 찾아 답변 해줘.
    만약 질문에 '모두', '전부'와 같은 단어가 포함되어 있다면 해당하는 정보를 문서에서 모두 최대한 찾아서 답변해줘.
    문서에 존재하지 않는 내용은 답하면 안돼.
    질문에 연도가 포함되어 있다면, 그 연도에 해당하는 정확한 데이터를 제공해줘.
    22Q3은 2022년도 3분기를 의미해. Q는 분기를 의미해.
    문서 내용과 관련 없거나 애매한 질문이면 '내용을 찾을 수 없습니다.'라고 답변해줘.
    문서에서 정보를 얻을 수 없다면 임의로 생성하지 말고 반드시 '문서에서 내용을 찾을 수 없습니다.'라고 답변해줘.
'''
# 메시지 템플릿 설정
messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("문맥: {context}\n\n질문: {question}\n\n한국어로 답변해 주세요:")
]
chat_prompt = ChatPromptTemplate.from_messages(messages)

# 체인 생성
chain = LLMChain(
    llm=llm,
    prompt=chat_prompt,
    output_parser=StrOutputParser(),
)

result = chain.run({
    "context": formatted_docs,
    "question": query,
})

print("답변 : ", result)
print("총 걸린 시간 : ", time.time() - start)

Starting the PDF loading process...
PDF loaded successfully.
Starting the text splitting process...
Text splitting completed.
=====chunks=====
Starting the embedding and vector store creation process...


c:\Users\jy901\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Vectorstore saved at ./MY_FAISS_INDEX_llama3
=====vectorstore loaded=====
요약 손익 계산서  (Non -GAAP 기준) 
(백만달러 ) 1Q22 2Q22  3Q22  4Q22  1Q23 2Q23 3Q23  4Q23  2022 2023 
매출액  10,697  11,158  10,928  11,387  9,456  9,268  8,442  8,665  44,170  35,831
매출액  22,611  24,273  23,531  33,566  44,200  
매출원가  10,244  8,599  9,255  14,262  18,635  
매출총이익  12,367  15,674  14,276  19,304  25,565  
(매출총이익률 , %) 54.7  64.6  60.7  57.5  57.8
8910111213
90100110120130140
'22.11 '23.04 '23.09
주가 (좌측)
EPS 추정치 (우측)(달러) (달러) Tech팀 
문준호  
Senior Analyst  
joonho.moon@samsung.com  
  
 
SUMMARY OF FINANCIAL DATA  
(백만달러) FY 4Q23  증감 (%) 
 Actual  Consensus  차이 (%)  전년 동기 대비 전분기  대비
1Q21 3Q21 1Q22 3Q22 1Q23 3Q23 1Q24E*
차량반도체 IoT 반도체(백만달러 )
GLOBAL EQUITY  2023. 11. 2 
Qualcomm  
 
Samsung Securities (Korea)  
www.samsungpop.com  4 
 부문별  breakdown (Non -GAAP 기준) 
(백만달러 ) 1Q22 2Q22  3Q22  4Q22  1Q23 2Q23  3Q23  4Q23  2022 2023 
매출액
===== formatted docs check =====
Formatted Docs:  요약 손익 계산서  (Non -GAAP 기준) 
(백만달러 )